# MultiIndex and Vectorization
A MultiIndex comes in handy in all situations where one has to deal with hierarchical data. 

In [2]:
import pandas as pd 
import numpy as np

In [21]:
df = (
    pd.read_csv(
    "https://raw.githubusercontent.com/jpatokal/openflights/master/data/airports.dat", 
    names=[
        "ix", 
        "name",
        "city",
        "country",
        "IATA",
        "ICAO",
        "lat",
        "long",
        "alt",
        "timezone",
        "dst",
        "tz_db",
        "type",
        "source"
    ],
    usecols=["IATA", "lat", "long"],
    index_col="IATA"
    )
    .sample(n=500, random_state=42)
    .groupby(level=0).first() # removes duplicates
    .sort_index()
)

In [22]:
df.head()

,lat,long
IATA,,
AAT,47.749886,88.085808
ABV,9.006790,7.263170
ACF,-27.570299,153.007996
ADF,37.731400,38.468899
AGU,21.705601,-102.318001


## Cross product

When it comes to producing the cross product of an input table with itself, you have multiple options. The first one below is taken from [Tom Augspurger's blogpost](https://tomaugspurger.github.io/modern-4-performance). It relies on a thinking in MultiIndex-terms: First create the MultiIndex, then reassign the existing DataFrames to the MultiIndex.

In [57]:
def cross_prod_1(df):
    idx = pd.MultiIndex.from_product([df.index, df.index], names=["orig", "dest"])
    pairs = pd.concat(
        [df.add_suffix('_1').reindex(idx, level="orig"),
        df.add_suffix('_2').reindex(idx, level="dest")],
        axis=1
    )
    return pairs

In [60]:
%%timeit
cross_prod_1(df)

83.9 ms ± 11.8 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


The second approach - the one that I came up with - uses pd.DataFrame.merge() to explicitly create a cross product of the original DataFrame with itself and then resets the index to a MultiIndex.

In [65]:
def cross_prod_2(df):
    df = df.reset_index()
    pairs = (
        df
        .merge(df, how="cross", suffixes=('_1', '_2'))
        .set_index(["IATA_1","IATA_2"]))
    pairs.index = pairs.index.rename(["orig", "dest"])
    return pairs

In [66]:
%%timeit
cross_prod_2(df)

63.7 ms ± 1.33 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [64]:
pd.testing.assert_frame_equal(cross_prod_1(df), cross_prod_2(df))

Apparently, both approaches are similar in performance, but I personally like Tom's approach a bit more. It is less obvious to the pandas newbie but a bit more elegant in the end.

## Vectorized calculations

Let's assume, we'd like to do a calculation on all rows of this MultiIndex dataframe that we just created. In general there are two paths we could take:

1. The iteration approach, which uses pd.DataFrame.apply(..., axis=1) or a for loop to apply the calculation to all rows
2. The broadcasting approach, which uses vectorization to apply the function to all rows.

So now in this example, we are for some reason interested in the sum of all elements of a row in the dataframe.

In [73]:
pairs = cross_prod_1(df)

In [108]:
def calc_iter(df):
    return df.apply(lambda row: sum(row), axis=1)

In [101]:
def calc_vector(df):
    row_sums =  np.sum(df, axis=1)
    return pd.Series(row_sums, index=df.index)

In [109]:
%%timeit
calc_iter(pairs)

1.55 s ± 78.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [103]:
%%timeit
calc_vector(pairs)

4.77 ms ± 1.31 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [110]:
pd.testing.assert_series_equal(
    calc_iter(pairs),
    calc_vector(pairs)
)